In [ ]:
!pip install datasets transformers torch pytube rouge-score
!pip install youtube-transcript-api
!pip install bert-extractive-summarizer

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
!pip install google.generativeai

In [ ]:
import torch
import os
import google.generativeai as genai
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
from summarizer import Summarizer,TransformerSummarizer
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer



In [ ]:
# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1

In [ ]:
def get_transcript(video_url):
  video_id = video_url.split("=")[-1]
  transcript_text =  YouTubeTranscriptApi.get_transcript(video_id)
  text = ""
  for i in transcript_text:
    text = text + " " + i['text']
  return text


In [ ]:
bert_model = Summarizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")


In [9]:
prompt="""You are Yotube video summarizer. You will be taking the transcript text
and summarizing the entire video and providing the important summary
within 250 words. Please provide the summary of the text given here:  """

genai.configure(api_key = os.getenv("GOOGLE_API_KEY"))

def generate_gemini_content(transcript_text,prompt):

    model=genai.GenerativeModel("gemini-pro")
    response=model.generate_content(prompt+transcript_text)
    return response.text

In [ ]:
# # Initialize pipelines with the device argument
summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn", device=device)



config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
summarizer_t5 = pipeline("summarization", model="t5-base", device=device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
def summarize_video(video_url):
    transcript = get_transcript(video_url)
    summary_bart = summarizer_bart(transcript, max_length=250, min_length=30, do_sample=False)[0]['summary_text']
    summary_t5 = summarizer_t5(transcript, max_length=250, min_length=30, do_sample=False)[0]['summary_text']
    summary_bert = ''.join(bert_model(transcript, max_length=250))

    summary_GPT2 = ''.join(GPT2_model(transcript, min_length=60))
    summary_gemini = generate_gemini_content(transcript,prompt)
    return summary_bart, summary_t5, summary_bert, summary_GPT2, summary_gemini


In [ ]:
summary_bart, summary_t5, summary_bert, summary_GPT2, summary_gemini = summarize_video("https://www.youtube.com/watch?v=KX3PbU8K4sg")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Example reference summary (for evaluation)
reference_summary = """Scientists have discovered a new species of orangutans on Indonesia's island of Sumatra, named the Tapanuli orangutan. The population differs from the two existing orangutan species found in Sumatra and the neighboring island of Borneo. The Tapanuli orangutans are considered a new species due to genetic, skeletal, and tooth differences. The orangutans live more lonely lives than other great apes, spending more time sleeping and eating fruit in the forest. The new species is the most endangered of all the great ape species, living within an area covering about 1,000 square kilometers. The population is highly vulnerable due to the environment they depend on being greatly threatened by development. If steps are not taken quickly, the new species could become extinct within our lifetime."""

# Calculate ROUGE scores
def evaluate_summary(summary, reference):
    scores = scorer.score(reference, summary)
    return scores

# Print evaluations
print("BART ROUGE Scores:\n", evaluate_summary(summary_bart, reference_summary))
print("t5 ROUGE Scores:\n", evaluate_summary(summary_t5, reference_summary))
print("BERT ROUGE Scores:\n", evaluate_summary(summary_bert, reference_summary))
print("GPT2 ROUGE Scores:\n", evaluate_summary(summary_GPT2, reference_summary))
print("gemini ROUGE Scores:\n", evaluate_summary(summary_gemini, reference_summary))




BART ROUGE Scores:
 {'rouge1': Score(precision=0.723404255319149, recall=0.2698412698412698, fmeasure=0.39306358381502887), 'rouge2': Score(precision=0.41304347826086957, recall=0.152, fmeasure=0.22222222222222224), 'rougeL': Score(precision=0.574468085106383, recall=0.21428571428571427, fmeasure=0.3121387283236994)}
t5 ROUGE Scores:
 {'rouge1': Score(precision=0.8611111111111112, recall=0.24603174603174602, fmeasure=0.38271604938271603), 'rouge2': Score(precision=0.6571428571428571, recall=0.184, fmeasure=0.2875), 'rougeL': Score(precision=0.75, recall=0.21428571428571427, fmeasure=0.3333333333333333)}
BERT ROUGE Scores:
 {'rouge1': Score(precision=0.6228070175438597, recall=0.5634920634920635, fmeasure=0.5916666666666667), 'rouge2': Score(precision=0.3893805309734513, recall=0.352, fmeasure=0.36974789915966383), 'rougeL': Score(precision=0.5087719298245614, recall=0.4603174603174603, fmeasure=0.48333333333333334)}
GPT2 ROUGE Scores:
 {'rouge1': Score(precision=0.6210526315789474, rec